# K-Means Clustering Analysis

This notebook implements the clustering component of the project.
We load the feature matrix, preprocess it, run K-Means with several values of *k*,
evaluate the clustering using different metrics, select the best configuration,
and visualize the resulting clusters using PCA.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings('ignore')


## 1. Load & Preprocess Data

We load the `.npy` feature matrix, standardize all features using `StandardScaler`,
and keep the full dataset for clustering. Standardization is important because
K-Means is distance-based and is sensitive to the scale of the features.


In [ ]:
# Load the feature matrix (ensure X_features.npy is in the same folder as this notebook)
X = np.load('X_features.npy')
print('Original shape:', X.shape)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print('Scaled shape:', X_scaled.shape)


## 2. K-Means Experiments

We test K-Means with different numbers of clusters:
`k = 10, 20, 30, 40, 50`.

For each configuration we compute three clustering quality metrics:
- **Silhouette Score** (higher is better)
- **Calinski–Harabasz Index** (higher is better)
- **Davies–Bouldin Index** (lower is better)


In [ ]:
k_values = [10, 20, 30, 40, 50]
results = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)

    sil = silhouette_score(X_scaled, labels)
    ch = calinski_harabasz_score(X_scaled, labels)
    db = davies_bouldin_score(X_scaled, labels)

    results.append([k, sil, ch, db])
    print(f'k={k} | Silhouette={sil:.4f} | CH={ch:.2f} | DB={db:.4f}')

results_df = pd.DataFrame(results, columns=['k', 'silhouette', 'calinski_harabasz', 'davies_bouldin'])
results_df


## 3. Compare Clustering Metrics

We now visualize how each metric behaves as a function of the number of clusters.
This helps to see trends and to justify the chosen value of *k*.


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(results_df['k'], results_df['silhouette'], marker='o')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs k')
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(results_df['k'], results_df['calinski_harabasz'], marker='o')
plt.xlabel('k (number of clusters)')
plt.ylabel('Calinski–Harabasz Index')
plt.title('Calinski–Harabasz Index vs k')
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(results_df['k'], results_df['davies_bouldin'], marker='o')
plt.xlabel('k (number of clusters)')
plt.ylabel('Davies–Bouldin Index')
plt.title('Davies–Bouldin Index vs k')
plt.show()


## 4. Select Best k

We select the best number of clusters using the **Silhouette Score** as the
primary criterion (higher is better). The other two metrics are used as
secondary checks.


In [ ]:
best_row = results_df.iloc[results_df['silhouette'].idxmax()]
best_k = int(best_row['k'])
print('Best k (by silhouette):', best_k)
best_row


## 5. Run Final K-Means Model and Save Labels

We now fit K-Means again using the selected number of clusters `best_k`.
We store the labels in a separate `.npy` file so they can be reused by
other parts of the project.


In [ ]:
best_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
best_labels = best_kmeans.fit_predict(X_scaled)

np.save('best_kmeans_labels.npy', best_labels)
print('Saved labels to best_kmeans_labels.npy')


## 6. PCA Visualization of Clusters

We reduce the standardized data to 2 dimensions using PCA and plot the
samples colored by their cluster assignment. This gives an intuitive view of
how the clusters are separated in a low-dimensional space.

Figures are saved in the `images/` folder for inclusion in the report.


In [ ]:
# Ensure images folder exists
os.makedirs('images', exist_ok=True)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=best_labels, s=10, cmap='tab20')
plt.title(f'PCA Visualization - K-Means (k={best_k})')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.tight_layout()
plt.savefig('images/pca_clusters.png', dpi=300)
plt.show()


## 7. Cluster Size Histogram

We also look at the distribution of samples across clusters. This helps us
understand whether some clusters are very small (potentially noise) or if the
clusters are relatively balanced.

This figure is also saved to the `images/` folder.


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x=best_labels)
plt.title(f'Cluster Size Distribution (k={best_k})')
plt.xlabel('Cluster Label')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('images/cluster_histogram.png', dpi=300)
plt.show()
